In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 4384, done.
remote: Counting objects: 100% (1330/1330), done.
remote: Compressing objects: 100% (519/519), done.
remote: Total 4384 (delta 896), reused 1177 (delta 797), pack-reused 3054
Receiving objects: 100% (4384/4384), 2.55 GiB | 36.61 MiB/s, done.
Resolving deltas: 100% (3378/3378), done.
Updating files: 100% (445/445), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 3.4 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/training-tuned-models/slim_bpr_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/slim_bpr_all.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/slim_bpr_train_validation.zip
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/.gitignore
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/README.md
/kaggle/input/training-tuned-models/Recsys_Challenge

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender, ScaledPureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_linear_comb.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.3408516511081212, 'beta': 0.1447198697205665, 'topK': 32}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_scaled.json', 'r') as params_file:
    params_scaled = json.load(params_file)
params_scaled

{'num_factors': 179,
 'scaling_items': 5.3298425368534826e-05,
 'scaling_users': 0.2791990262721904}

In [21]:
folds = []

for i in tqdm.auto.tqdm(range(10)):
    fold_dict = {}
    fold_dict['URM_train'] = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}/URM_train.npz')
    fold_dict['URM_valid'] = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}/URM_valid.npz')
    fold_dict['Evaluator'] = EvaluatorHoldout(fold_dict['URM_valid'], cutoff_list=[10])
    
    recommender_item = ItemKNNCFRecommender(fold_dict['URM_train'])
    recommender_item.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'item_knn.zip')
    fold_dict['ItemKNN'] = recommender_item

    recommender_user = UserKNNCFRecommender(fold_dict['URM_train'])
    recommender_user.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'user_knn.zip')
    fold_dict['UserKNN'] = recommender_user

    recommender_rp3 = RP3betaRecommender(fold_dict['URM_train'])
    recommender_rp3.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'rp3.zip')
    fold_dict['RP3'] = recommender_rp3

    recommender_p3a = P3alphaRecommender(fold_dict['URM_train'])
    recommender_p3a.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'p3a.zip')
    fold_dict['P3alpha'] = recommender_p3a

    recommender_pure_svd = PureSVDRecommender(fold_dict['URM_train'])
    recommender_pure_svd.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'pure_svd.zip')
    fold_dict['PureSVD'] = recommender_pure_svd

    recommender_pure_svd_item = PureSVDItemRecommender(fold_dict['URM_train'])
    recommender_pure_svd_item.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'item_pure_svd.zip')
    fold_dict['PureSVDItem'] = recommender_pure_svd_item

    recommender_scaled_svd = ScaledPureSVDRecommender(fold_dict['URM_train'])
    recommender_scaled_svd.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'pure_svd_scaled.zip')
    fold_dict['ScaledPureSVD'] = recommender_scaled_svd

    recommender_nmf = NMFRecommender(fold_dict['URM_train'])
    recommender_nmf.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'nmf.zip')
    fold_dict['NMF'] = recommender_nmf

    recommender_slim_bpr = SLIM_BPR_Python(fold_dict['URM_train'])
    recommender_slim_bpr.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'slim_bpr.zip')
    fold_dict['SlimBPR'] = recommender_slim_bpr

    recommender_slim = SLIMElasticNetRecommender(fold_dict['URM_train'])
    recommender_slim.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'slim_elastic.zip')
    fold_dict['SlimElastic'] = recommender_slim
    
    recommender_ease_r = EASE_R_Recommender(fold_dict['URM_train'])
    recommender_ease_r.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'ease_r.zip')
    fold_dict['EaseR'] = recommender_ease_r   
    
    folds.append(fold_dict)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 4017 (30.8%) Users that have less than 1 test interactions
ItemKNNCFRecommender: URM Detected 490 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 176 ( 0.8%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0item_knn.zip'
ItemKNNCFRecommender: Loading complete
UserKNNCFRecommender: URM Detected 490 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 176 ( 0.8%) items with no interactions.
UserKNNCFRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0user_knn.zip'
UserKNNCFRecommender: Loading complete
RP3betaRecommender: URM Detected 490 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 176 ( 0.8%) items with no interactions.
RP3betaRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0rp3.zip'
RP3betaRecommender: Loading complete


In [22]:
def objective(trial):
    weights = [
        trial.suggest_float('SlimElastic', 1e-2, 1, log=True),
        trial.suggest_float('ItemKNN', 1e-3, 1, log=True),
        trial.suggest_float('RP3', 1e-3, 1, log=True),
        trial.suggest_float('P3alpha', 1e-3, 1, log=True),
        # trial.suggest_float('IALS', 1e-4, 1, log=True),
        trial.suggest_float('EaseR', 1e-4, 1, log=True),
        trial.suggest_float('UserKNN', 1e-4, 1, log=True),
        trial.suggest_float('PureSVD', 1e-5, 0.2, log=True),
        trial.suggest_float('PureSVDItem', 1e-5, 0.2, log=True),
        trial.suggest_float('ScaledPureSVD', 1e-5, 0.2, log=True),
        trial.suggest_float('NMF', 1e-5, 0.2, log=True),
        trial.suggest_float('SlimBPR', 1e-6, 0.2, log=True),
    ]
    
    results = []
    for fold_dict in folds:
        recommenders = [
            fold_dict['SlimElastic'], fold_dict['ItemKNN'], fold_dict['RP3'], fold_dict['P3alpha'],
            fold_dict['EaseR'], fold_dict['UserKNN'], fold_dict['PureSVD'], fold_dict['PureSVDItem'],
            fold_dict['ScaledPureSVD'], fold_dict['NMF'], fold_dict['SlimBPR']
        ]
    
        linear_combinator = LinearCombinationRecommender(fold_dict['URM_train'], recommenders, weights)
        result_df, _ = fold_dict['Evaluator'].evaluateRecommender(linear_combinator)
        results.append(result_df.loc[10, 'MAP'])

    return np.mean(results)

In [23]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_merging.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [24]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_linear_11_models_cv', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=50)

[I 2023-12-11 16:41:22,136] A new study created in RDB with name: hyperparameters_tuning_linear_11_models_cv


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 41.14 sec. Users per second: 219
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 40.80 sec. Users per second: 220
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 41.08 sec. Users per second: 220
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 38.75 sec. Users per second: 233
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-11 16:47:47,051] Trial 0 finished with value: 0.02044011716425489 and parameters: {'SlimElastic': 0.20525169841574817, 'ItemKNN': 0.017184601001655486, 'RP3': 0.012269027609756424, 'P3alpha': 0.01627075992704425, 'EaseR': 0.00034513637497068456, 'UserKNN': 0.001013050738285963, 'PureSVD': 2.0769800744715608e-05, 'PureSVDItem': 2.439038052756631e-05, 'ScaledPureSVD': 1.9450278998639816e-05, 'NMF': 0.0006219686732049527, 'SlimBPR': 0.0005301778678521846}. Best is trial 0 with value: 0.02044011716425489.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 41.85 sec. Users per second: 215
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 41.66 sec. Users per second: 216
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 40.86 sec. Users per second: 221
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 39.33 sec. Users per second: 230
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-11 16:54:08,590] Trial 1 finished with value: 0.019673883630604433 and parameters: {'SlimElastic': 0.10846217924358813, 'ItemKNN': 0.044918614253207576, 'RP3': 0.7456599792524908, 'P3alpha': 0.0046923596600839, 'EaseR': 0.0003995955531702542, 'UserKNN': 0.002686685684734256, 'PureSVD': 0.08878114367848942, 'PureSVDItem': 0.00045325633826039997, 'ScaledPureSVD': 0.08940287368339057, 'NMF': 0.06130880331677384, 'SlimBPR': 0.0006167596556920276}. Best is trial 0 with value: 0.02044011716425489.


...

[I 2023-12-11 21:53:39,390] Trial 47 finished with value: 0.02051426573886495 and parameters: {'SlimElastic': 0.059993255677212844, 'ItemKNN': 0.009970387684723123, 'RP3': 0.0067659567571607656, 'P3alpha': 0.008533401632867555, 'EaseR': 0.00010056576727955495, 'UserKNN': 0.000627747692103972, 'PureSVD': 4.056448927750896e-05, 'PureSVDItem': 0.00018960737692554338, 'ScaledPureSVD': 5.891280685577146e-05, 'NMF': 5.765519975067059e-05, 'SlimBPR': 0.00039306796196581086}. Best is trial 47 with value: 0.02051426573886495.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 43.29 sec. Users per second: 208
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 43.03 sec. Users per second: 209
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 42.99 sec. Users per second: 210
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 41.16 sec. Users per second: 220
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-11 22:00:16,784] Trial 48 finished with value: 0.020364584083081918 and parameters: {'SlimElastic': 0.06050064524288015, 'ItemKNN': 0.011812484942028226, 'RP3': 0.006264143685849365, 'P3alpha': 0.008365564804829733, 'EaseR': 0.0001233914193341638, 'UserKNN': 0.0018230949735971607, 'PureSVD': 7.19225085176146e-05, 'PureSVDItem': 0.0004656826963860901, 'ScaledPureSVD': 1.5054175716228782e-05, 'NMF': 6.744173635075155e-05, 'SlimBPR': 0.00029849275625161084}. Best is trial 47 with value: 0.02051426573886495.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 43.88 sec. Users per second: 205
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 43.38 sec. Users per second: 207
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 45.42 sec. Users per second: 199
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 41.42 sec. Users per second: 218
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-11 22:06:58,149] Trial 49 finished with value: 0.020578909529668624 and parameters: {'SlimElastic': 0.06487360728205781, 'ItemKNN': 0.008800922355191625, 'RP3': 0.025247969471010993, 'P3alpha': 0.009030548614730655, 'EaseR': 0.00010025324766680126, 'UserKNN': 0.0007382317678535077, 'PureSVD': 4.40785275619992e-05, 'PureSVDItem': 0.00016329267724967647, 'ScaledPureSVD': 3.947366691619507e-05, 'NMF': 3.257769097665972e-05, 'SlimBPR': 7.916627412612215e-05}. Best is trial 49 with value: 0.020578909529668624.


In [25]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [26]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [27]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [28]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_lr_comb_11_best_models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_lr_comb_11_best_models.json', 
            'tuning_results/best_params_hybrid_lr_comb_11_best_models.json', 
            'Linear combination of 11 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/linear-combination/best_params_hybrid_lr_comb_11_best_models.json', 
        '/kaggle/working/best_params_hybrid_lr_comb_11_best_models.json'
    )

In [29]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_merging.db', 
        'Linear combination tuning db updated results (from kaggle notebook)'
    )

In [30]:
with open('/kaggle/working/best_params_hybrid_lr_comb_11_best_models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'SlimElastic': 0.06487360728205781,
 'ItemKNN': 0.008800922355191625,
 'RP3': 0.025247969471010993,
 'P3alpha': 0.009030548614730655,
 'EaseR': 0.00010025324766680126,
 'UserKNN': 0.0007382317678535077,
 'PureSVD': 4.40785275619992e-05,
 'PureSVDItem': 0.00016329267724967647,
 'ScaledPureSVD': 3.947366691619507e-05,
 'NMF': 3.257769097665972e-05,
 'SlimBPR': 7.916627412612215e-05}

In [31]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_scaled_svd = ScaledPureSVDRecommender(URM_all)
recommender_scaled_svd.fit(**params_scaled) 

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

# recommender_ials = IALSRecommender(URM_all)
# recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4137.73 column/sec. Elapsed time 5.40 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 3924.61 column/sec. Elapsed time 3.32 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2785.73 column/sec. Elapsed time 8.02 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3116.64 column/sec. Elapsed time 7.17 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [32]:
weights = [
    params['SlimElastic'],
    params['ItemKNN'],
    params['RP3'],
    params['P3alpha'],
    # params['IALS'],
    params['EaseR'],
    params['UserKNN'],
    params['PureSVD'],
    params['PureSVDItem'],
    params['ScaledPureSVD'],
    params['NMF'],
    params['SlimBPR']
]

recommenders = [
    recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ease_r,
    recommender_user, recommender_pure_svd, recommender_pure_svd_item, recommender_scaled_svd, 
    recommender_nmf, recommender_slim_bpr
]

In [33]:
linear_combinator = LinearCombinationRecommender(URM_all, recommenders, weights)
recommendations = linear_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_linear_comb_11_best_models.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,36 101 123 506 403 515 694 1546 52 1422
1,2,1095 47 12 50 28 1522 11 656 102 196
2,3,59 857 259 4252 2172 648 536 584 956 1281
3,4,28 249 50 139 171 7 146 314 254 5
4,5,1570 77 170 131 95 1511 5138 471 116 1220
